In [ ]:
from openai import OpenAI
from openai import OpenAI
import os 
os.environ['OPENAI_API_KEY'] = ''
# client = OpenAI(
#   organization='org-T4ew5eiTxSpJrQVcBLLZQhTF',
#   project='proj_GJ4tiZkw4JfcR6nEhgZlDI39',
# )
client = OpenAI()

batchs  = client.batches.list(limit=10)

In [4]:
for batch in batchs.data:
    print(batch)

Batch(id='batch_JceTuY2k6VqjuyJCeM8TftTe', completion_window='24h', created_at=1715061295, endpoint='/v1/chat/completions', input_file_id='file-t0GxnzItsJlWNko8RFIoUM8l', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-3.5-turbo in organization org-T4ew5eiTxSpJrQVcBLLZQhTF. Limit: 200,000 enqueued tokens. Please try again once some in_progress batches have been completed.', param=None)], object='list'), expired_at=None, expires_at=1715147695, failed_at=1715061295, finalizing_at=None, in_progress_at=None, metadata={}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
Batch(id='batch_5b3wpapyO3gnxtQOwh6ZI67T', completion_window='24h', created_at=1714925635, endpoint='/v1/chat/completions', input_file_id='file-3v6lcbEyMWptOUg7ea78zlNE', object='batch', status='completed', ca

In [9]:
test_file = client.files.content('file-AQ5alMiZNTbicudhMbBUWFHP').write_to_file(o)

In [ ]:
import glob 
import os
import time
import threading
from pathlib import Path

for file in glob.glob('patch_api\cross\patch_batch_cross_project*.jsonl'):
    output_file = f'output_{file}'
    if os.path.exists(output_file):
        continue
    file_openai = client.files.create(
        file=Path(file),
        purpose="chat",
    )
    file_id = file_openai.id 
    tmp_batch = client.batches.create(
        input_file_id= file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
        "description": "nightly eval job"
            }
    )
    batch_id = tmp_batch.id
    while True:
        time.sleep(1000*60) #check status batch every 60s
        try:
            batch = client.batches.retrieve(batch_id)
            if batch.status == 'completed':
                client.files.content(batch.output_file_id).write_to_file(output_file)
                print('='*33)
                print(f'done: {file}')
                print(f'output: {output_file}')
                break
        except Exception as e:
            print(e)